# In this notebook, first different neighborhoods of vancouver are visualized and then the neighborhoods are clustered based on the number of bakeries they have to identify the possible best location to open a new bakery
## The KML file containing Vancouver neighborhood coordinates was downloaded from   
https://data.vancouver.ca/datacatalogue/localAreaBoundary.htm
## and then converted to json by a convreter 
### first, we import the libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: done

## Package Plan ##

  environment location: /anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.7.10               |           py37_0         3.0 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0
  geo

__Lets look open the geojson file and look at the data__

In [110]:
import json 

with open("van.geojson") as f:
    data = json.load(f)

In [49]:
neighborhoods_data = data['features']

let's look at the first entry as an example

In [111]:
neighborhoods_data[0]

{'type': 'Feature',
 'properties': {'Name': 'Arbutus-Ridge', 'description': None},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-123.16407742426055, 49.257482642589025],
    [-123.16385512340415, 49.257459575262544],
    [-123.16363263565299, 49.25744846897926],
    [-123.1626484965671, 49.2574328611031],
    [-123.1602805149013, 49.25739570015086],
    [-123.15791253524193, 49.25735804099439],
    [-123.15554331084444, 49.25731979537834],
    [-123.15389870447338, 49.25729321891287],
    [-123.1535964600301, 49.25728813515067],
    [-123.15317533708561, 49.2572816793889],
    [-123.15289337947901, 49.25727728636099],
    [-123.15257847924624, 49.2572471251311],
    [-123.15276035999341, 49.2518491408516],
    [-123.15276333023267, 49.251710099017025],
    [-123.15275530985528, 49.25157116164505],
    [-123.15273426191207, 49.251433140831985],
    [-123.15270335939522, 49.251296122311],
    [-123.15265931647919, 49.25116011041606],
    [-123.15260530890853, 49.2510260902112],
 

__json to dataframe__

In [51]:
from pandas.io.json import json_normalize
df = json_normalize(data["features"])

In [52]:
df

,geometry.coordinates,geometry.type,properties.Name,properties.description,type
0,"[[[-123.16407742426055, 49.257482642589025], [...",Polygon,Arbutus-Ridge,None,Feature
1,"[[[-123.15528417444278, 49.23456853651252], [-...",Polygon,Shaughnessy,None,Feature
2,"[[[-123.100028480465, 49.272776231346114], [-1...",Polygon,Mount Pleasant,None,Feature
3,"[[[-123.12652476577578, 49.280837980353084], [...",Polygon,Downtown,None,Feature
4,"[[[-123.1362459884374, 49.275841011656695], [-...",Polygon,West End,None,Feature
5,"[[[-123.09993329132439, 49.27492383793663], [-...",Polygon,Strathcona,None,Feature
6,"[[[-123.05649003189711, 49.29187700607315], [-...",Polygon,Grandview-Woodland,None,Feature
7,"[[[-123.05660149713069, 49.26214915403828], [-...",Polygon,Hastings-Sunrise,None,Feature
8,"[[[-123.10565550196236, 49.23315910836121], [-...",Polygon,Riley Park,None,Feature
9,"[[[-123.11619334368275, 49.23354308170746], [-...",Polygon,South Cambie,None,Feature


__since each neighborhood is a polygon, the coordinates given are related to each cornerof the polygon, in order to represent the neighborhood with only one point, we have to do a couple of steps:__  
first, we get rid of the extra brackets in geometry.coordinates by flattening:  

In [53]:
for index, row in df.iterrows():
    flattened  = [val for sublist in df.loc[index, "geometry.coordinates"] for val in sublist]
    df.loc[index, "geometry.coordinates"]=flattened
    print (len(df.loc[index, "geometry.coordinates"]))


226
205
685
2533
790
1519
668
2095
185
169
82
1343
823
1235
1955
1816
2620
817
1431
1115
198
295


In [54]:
df

,geometry.coordinates,geometry.type,properties.Name,properties.description,type
0,"[[-123.16407742426055, 49.257482642589025], [-...",Polygon,Arbutus-Ridge,None,Feature
1,"[[-123.15528417444278, 49.23456853651252], [-1...",Polygon,Shaughnessy,None,Feature
2,"[[-123.100028480465, 49.272776231346114], [-12...",Polygon,Mount Pleasant,None,Feature
3,"[[-123.12652476577578, 49.280837980353084], [-...",Polygon,Downtown,None,Feature
4,"[[-123.1362459884374, 49.275841011656695], [-1...",Polygon,West End,None,Feature
5,"[[-123.09993329132439, 49.27492383793663], [-1...",Polygon,Strathcona,None,Feature
6,"[[-123.05649003189711, 49.29187700607315], [-1...",Polygon,Grandview-Woodland,None,Feature
7,"[[-123.05660149713069, 49.26214915403828], [-1...",Polygon,Hastings-Sunrise,None,Feature
8,"[[-123.10565550196236, 49.23315910836121], [-1...",Polygon,Riley Park,None,Feature
9,"[[-123.11619334368275, 49.23354308170746], [-1...",Polygon,South Cambie,None,Feature


second, we have to average all longitudes as well as all latitudes for all coordinates in each neighborhood 

In [80]:
for index, row in df.iterrows():
    # Store lengths of each list and their cumulative and entire summations
    lens = np.array([len(i) for i in df.loc[index, "geometry.coordinates"]])
        # Only loop to get lengths
    C = lens.cumsum()
    N = lens.sum()

    # Create ID array such that the first element of each list is 0, 
    # the second element as 1 and so on. This is needed in such a format
    # for use with bincount later on.
    shifts_arr = np.ones(N,dtype=int)
    shifts_arr[C[:-1]] = -lens[:-1]+1
    id_arr = shifts_arr.cumsum()-1

# Use bincount to get the summations and thus the 
# averages across all lists based on their positions. 
    avg_out = np.bincount(id_arr,np.concatenate(df.loc[index, "geometry.coordinates"]))/np.bincount(id_arr)
    print (avg_out)
    df.loc[index, "geometry.coordinates"]=avg_out

[-123.1632582   49.2471774]
[-123.14126023   49.24673421]
[-123.10659313   49.27009858]
[-123.11793503   49.28192248]
[-123.14048572   49.28226329]
[-123.08644454   49.28545281]
[-123.06356153   49.28610422]
[-123.03934544   49.29086557]
[-123.11007853   49.2437625 ]
[-123.11919685   49.24441638]
[-123.12197253   49.22799009]
[-123.0404706    49.20788991]
[-123.06441146   49.20973089]
[-123.09174176   49.20763859]
[-123.13587922   49.2029247 ]
[-123.16232903   49.21351184]
[-123.19151958   49.22377556]
[-123.20090321   49.27325158]
[-123.15936831   49.27400278]
[-123.13018094   49.26900375]
[-123.06861264   49.2455679 ]
[-123.0424587    49.24223783]


In [81]:
df

,geometry.coordinates,geometry.type,properties.Name,properties.description,type
0,"[-123.16325820137328, 49.24717739977333]",Polygon,Arbutus-Ridge,None,Feature
1,"[-123.14126023363018, 49.246734209389174]",Polygon,Shaughnessy,None,Feature
2,"[-123.10659312564293, 49.2700985847883]",Polygon,Mount Pleasant,None,Feature
3,"[-123.11793502974928, 49.281922483293016]",Polygon,Downtown,None,Feature
4,"[-123.14048571556758, 49.28226328645321]",Polygon,West End,None,Feature
5,"[-123.0864445424009, 49.28545281475496]",Polygon,Strathcona,None,Feature
6,"[-123.0635615273131, 49.286104223391774]",Polygon,Grandview-Woodland,None,Feature
7,"[-123.03934543723832, 49.29086556843003]",Polygon,Hastings-Sunrise,None,Feature
8,"[-123.11007853183315, 49.243762504756894]",Polygon,Riley Park,None,Feature
9,"[-123.11919684712973, 49.24441638203378]",Polygon,South Cambie,None,Feature


__lets drop unnecessary columns for our purpose__

In [84]:
df.drop(["geometry.type","type","properties.description"],axis=1,inplace=True)

In [85]:
df

,geometry.coordinates,properties.Name
0,"[-123.16325820137328, 49.24717739977333]",Arbutus-Ridge
1,"[-123.14126023363018, 49.246734209389174]",Shaughnessy
2,"[-123.10659312564293, 49.2700985847883]",Mount Pleasant
3,"[-123.11793502974928, 49.281922483293016]",Downtown
4,"[-123.14048571556758, 49.28226328645321]",West End
5,"[-123.0864445424009, 49.28545281475496]",Strathcona
6,"[-123.0635615273131, 49.286104223391774]",Grandview-Woodland
7,"[-123.03934543723832, 49.29086556843003]",Hastings-Sunrise
8,"[-123.11007853183315, 49.243762504756894]",Riley Park
9,"[-123.11919684712973, 49.24441638203378]",South Cambie


__now we put the average latitudes and longitudes in separate lists

In [91]:
new1=[]
new2=[]
for index, row in df.iterrows():
    new1.append(df.loc[index, "geometry.coordinates"][0])
    new2.append(df.loc[index, "geometry.coordinates"][1])

In [92]:
new1

[-123.16325820137328,
 -123.14126023363018,
 -123.10659312564293,
 -123.11793502974928,
 -123.14048571556758,
 -123.0864445424009,
 -123.0635615273131,
 -123.03934543723832,
 -123.11007853183315,
 -123.11919684712973,
 -123.12197253339349,
 -123.040470604042,
 -123.06441146314864,
 -123.09174176336481,
 -123.13587922126035,
 -123.16232902677159,
 -123.19151957933396,
 -123.20090321250989,
 -123.1593683136649,
 -123.13018093627367,
 -123.06861264078012,
 -123.04245870165096]

__adding those lists to our dataframe__

In [93]:
df['longitude'] = new1
df['latitude'] = new2

In [94]:
df

,geometry.coordinates,properties.Name,longitude,latitude
0,"[-123.16325820137328, 49.24717739977333]",Arbutus-Ridge,-123.163258,49.247177
1,"[-123.14126023363018, 49.246734209389174]",Shaughnessy,-123.141260,49.246734
2,"[-123.10659312564293, 49.2700985847883]",Mount Pleasant,-123.106593,49.270099
3,"[-123.11793502974928, 49.281922483293016]",Downtown,-123.117935,49.281922
4,"[-123.14048571556758, 49.28226328645321]",West End,-123.140486,49.282263
5,"[-123.0864445424009, 49.28545281475496]",Strathcona,-123.086445,49.285453
6,"[-123.0635615273131, 49.286104223391774]",Grandview-Woodland,-123.063562,49.286104
7,"[-123.03934543723832, 49.29086556843003]",Hastings-Sunrise,-123.039345,49.290866
8,"[-123.11007853183315, 49.243762504756894]",Riley Park,-123.110079,49.243763
9,"[-123.11919684712973, 49.24441638203378]",South Cambie,-123.119197,49.244416


In [96]:
df.drop(["geometry.coordinates"],axis=1,inplace=True)

In [98]:
df.columns = ['Neighborhood', 'longitude', 'latitude']

In [99]:
df

,Neighborhood,longitude,latitude
0,Arbutus-Ridge,-123.163258,49.247177
1,Shaughnessy,-123.141260,49.246734
2,Mount Pleasant,-123.106593,49.270099
3,Downtown,-123.117935,49.281922
4,West End,-123.140486,49.282263
5,Strathcona,-123.086445,49.285453
6,Grandview-Woodland,-123.063562,49.286104
7,Hastings-Sunrise,-123.039345,49.290866
8,Riley Park,-123.110079,49.243763
9,South Cambie,-123.119197,49.244416


In [102]:
cols = df.columns.tolist()
cols=['Neighborhood', 'latitude', 'longitude']
df = df[cols]
df

,Neighborhood,latitude,longitude
0,Arbutus-Ridge,49.247177,-123.163258
1,Shaughnessy,49.246734,-123.141260
2,Mount Pleasant,49.270099,-123.106593
3,Downtown,49.281922,-123.117935
4,West End,49.282263,-123.140486
5,Strathcona,49.285453,-123.086445
6,Grandview-Woodland,49.286104,-123.063562
7,Hastings-Sunrise,49.290866,-123.039345
8,Riley Park,49.243763,-123.110079
9,South Cambie,49.244416,-123.119197


__finding geo-coordination of vancouver via geopy__

In [100]:
address = 'Vancouver, BC'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


__finally creating a map with Vancouver Neighborhoods via folium__

In [109]:
# create map using latitude and longitude values
map_van = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['latitude'], df['longitude'], df['Neighborhood']):
   # label = {}.format(neighborhood)
    label = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_van)  
    
map_van

## Let's use Foursquare developer to explore the neighborhoods

In [112]:
CLIENT_ID = 'ASRI2J3DNFY1NYGNCINKNUFVWMU2JAHIQ52BGIZZ2ZRI1ZAG' # your Foursquare ID
CLIENT_SECRET = 'JCTR5JLVL31YS4PB0BM2BGTVIKBN2DJW5HR4LOORMI2ZEIGB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ASRI2J3DNFY1NYGNCINKNUFVWMU2JAHIQ52BGIZZ2ZRI1ZAG
CLIENT_SECRET:JCTR5JLVL31YS4PB0BM2BGTVIKBN2DJW5HR4LOORMI2ZEIGB


In [118]:
neighborhood_latitude =df.loc[3, 'latitude'] # neighborhood latitude value
neighborhood_longitude =df.loc[3, 'longitude'] # neighborhood longitude value

neighborhood_name = df.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Downtown are 49.281922483293016, -123.11793502974928.


In [119]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ASRI2J3DNFY1NYGNCINKNUFVWMU2JAHIQ52BGIZZ2ZRI1ZAG&client_secret=JCTR5JLVL31YS4PB0BM2BGTVIKBN2DJW5HR4LOORMI2ZEIGB&v=20180605&ll=49.281922483293016,-123.11793502974928&radius=500&limit=100'

In [120]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d45faed9ba3e5002d9f1e61'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Vancouver',
  'headerFullLocation': 'Downtown Vancouver, Vancouver',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 157,
  'suggestedBounds': {'ne': {'lat': 49.28642248779302,
    'lng': -123.11104963379857},
   'sw': {'lat': 49.277422478793014, 'lng': -123.12482042569998}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d5ec8ce29ef236ae0cb9059',
       'name': 'Rosewood Hotel Georgia',
       'location': {'address': '801 W Georgia St',
        'crossStreet': 'at Howe St',
        'lat': 49.28342914255638,
        'lng': -123.1189107326352,
 

In [121]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [123]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Rosewood Hotel Georgia,Hotel,49.283429,-123.118911
1,Hawksworth Restaurant,Lounge,49.283362,-123.119462
2,Gotham Steakhouse & Cocktail Bar,Steakhouse,49.282830,-123.115865
3,L'Hermitage,Hotel,49.280139,-123.117480
4,Medina Café,Breakfast Spot,49.280565,-123.116859
5,Disney store,Toy / Game Store,49.281689,-123.119850
6,Tacofino Cantina,Food Truck,49.282889,-123.118725
7,The Keg Steakhouse + Bar,Steakhouse,49.283452,-123.116174
8,Commodore Ballroom,Concert Hall,49.280686,-123.120510
9,Abercrombie & Fitch,Clothing Store,49.282274,-123.118685


In [124]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [127]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['latitude'], df['longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [128]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'latitude', 'longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1838, 7)


,Neighborhood,latitude,longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Arbutus-Ridge,49.247177,-123.163258,The Patty Shop,49.250680,-123.167916,Caribbean Restaurant
1,Arbutus-Ridge,49.247177,-123.163258,Butter Baked Goods,49.242209,-123.170381,Bakery
2,Arbutus-Ridge,49.247177,-123.163258,The Arbutus Club,49.248507,-123.152152,Event Space
3,Arbutus-Ridge,49.247177,-123.163258,La Buca,49.250549,-123.167933,Italian Restaurant
4,Arbutus-Ridge,49.247177,-123.163258,Sweet Obsession Cakes & Pastries,49.257756,-123.165314,Dessert Shop


## Let's check how many venues were returned for each neighorhood

In [129]:
venues_df.groupby(["Neighborhood"]).count()

,latitude,longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Arbutus-Ridge,100,100,100,100,100,100
Downtown,100,100,100,100,100,100
Dunbar-Southlands,17,17,17,17,17,17
Fairview,100,100,100,100,100,100
Grandview-Woodland,100,100,100,100,100,100
Hastings-Sunrise,98,98,98,98,98,98
Kensington-Cedar Cottage,100,100,100,100,100,100
Kerrisdale,48,48,48,48,48,48
Killarney,41,41,41,41,41,41


## Let's find out how many unique categories can be curated from all the returned venues


In [130]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 215 uniques categories.


In [131]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Caribbean Restaurant', 'Bakery', 'Event Space',
       'Italian Restaurant', 'Dessert Shop', 'Pizza Place', 'Park',
       'Japanese Restaurant', 'Portuguese Restaurant',
       'Chinese Restaurant', 'Baseball Field', 'Grocery Store', 'Café',
       'Coffee Shop', 'Tennis Court', 'Tea Room', 'Breakfast Spot',
       'Sandwich Place', 'Yoga Studio', 'Diner', 'Pharmacy',
       'Gym / Fitness Center', 'Farmers Market',
       'Vegetarian / Vegan Restaurant', 'Spa', 'Indie Movie Theater',
       'Pet Store', 'Restaurant', 'Ice Cream Shop', 'Thai Restaurant',
       'Vietnamese Restaurant', 'Bagel Shop', 'Ramen Restaurant',
       'Indian Restaurant', 'Shopping Mall', 'Deli / Bodega',
       'Fast Food Restaurant', 'Frozen Yogurt Shop', 'Wine Shop',
       'Sushi Restaurant', 'Furniture / Home Store', 'Noodle House',
       'Mediterranean Restaurant', 'Mexican Restaurant', 'Boutique',
       'Cosmetics Shop', 'Bank', 'Gym', 'French Restaurant',
       'Liquor Store'], dtype=object)

## Analyze each neighborhood

In [134]:
# one hot encoding
van_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
van_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [van_onehot.columns[-1]] + list(van_onehot.columns[:-1])
van_onehot = van_onehot[fixed_columns]

print(van_onehot.shape)
van_onehot.head()

(1838, 216)


,Neighborhoods,Accessories Store,Airport Service,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dog Run,Donut Shop,Electronics Store,Event Space,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Heliport,Historic Site,Hobby Shop,Hockey Arena,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trade School,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Arbutu

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [136]:
van_grouped = van_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(van_grouped.shape)
van_grouped

(22, 216)


,Neighborhoods,Accessories Store,Airport Service,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dog Run,Donut Shop,Electronics Store,Event Space,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Heliport,Historic Site,Hobby Shop,Hockey Arena,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Night Market,Nightclub,Noodle House,Office,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Soccer Stadium,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trade School,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Arbutus-Ridge,0.00,0.000000,0.000000,0.00,0.000000,0.010000,0.00,0.000000,0.00,0.01,0.020000,0.020000,0.000000,0.01,0.00,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.01,0.00,0.00,0.020000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.030000,0.00,0.000000,0.000000,0.01,0.00,0.00,0.020000,0.00,0.000000,0.00,0.130000,0.00,0.000000,0.00,0.010000,0.01,0.00,0.00,0.00000,0.01,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.030000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.01,0.010000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.00,0.000000,0.050000,0.020000,0.010000,0.010000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.010000,0.010000,0.01,0.000000,0.000000,0.030000,0.040000,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.010000,0.0

In [167]:
len(van_grouped[van_grouped["Bakery"] > 0])


20

In [149]:
#Create a new DataFrame for bakery data only

van_bakery = van_grouped[["Neighborhoods","Bakery"]]


In [150]:
van_bakery.head()


,Neighborhoods,Bakery
0,Arbutus-Ridge,0.02
1,Downtown,0.03
2,Dunbar-Southlands,0.00
3,Fairview,0.05
4,Grandview-Woodland,0.03


In [152]:
# set number of clusters
kclusters = 3

van_clustering = van_bakery.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(van_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 1, 0, 0, 0, 0, 0, 1], dtype=int32)

In [153]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
van_merged = van_bakery.copy()

# add clustering labels
van_merged["Cluster Labels"] = kmeans.labels_

In [154]:
van_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
van_merged.head()

,Neighborhood,Bakery,Cluster Labels
0,Arbutus-Ridge,0.02,0
1,Downtown,0.03,0
2,Dunbar-Southlands,0.00,2
3,Fairview,0.05,1
4,Grandview-Woodland,0.03,0


In [155]:
#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
van_merged = van_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(van_merged.shape)
van_merged.head() # check the last columns!

(22, 5)


,Neighborhood,Bakery,Cluster Labels,latitude,longitude
0,Arbutus-Ridge,0.02,0,49.247177,-123.163258
1,Downtown,0.03,0,49.281922,-123.117935
2,Dunbar-Southlands,0.00,2,49.223776,-123.191520
3,Fairview,0.05,1,49.269004,-123.130181
4,Grandview-Woodland,0.03,0,49.286104,-123.063562


In [156]:
# sort the results by Cluster Labels
print(van_merged.shape)
van_merged.sort_values(["Cluster Labels"], inplace=True)
van_merged

(22, 5)


,Neighborhood,Bakery,Cluster Labels,latitude,longitude
0,Arbutus-Ridge,0.020000,0,49.247177,-123.163258
1,Downtown,0.030000,0,49.281922,-123.117935
18,Sunset,0.016667,0,49.207639,-123.091742
17,Strathcona,0.030000,0,49.285453,-123.086445
4,Grandview-Woodland,0.030000,0,49.286104,-123.063562
5,Hastings-Sunrise,0.030612,0,49.290866,-123.039345
6,Kensington-Cedar Cottage,0.020000,0,49.245568,-123.068613
7,Kerrisdale,0.020833,0,49.213512,-123.162329
8,Killarney,0.024390,0,49.207890,-123.040471
16,South Cambie,0.020000,0,49.244416,-123.119197


## Creating a map of clusters

In [158]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(van_merged['latitude'], van_merged['longitude'], van_merged['Neighborhood'], van_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [159]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examining clusters:

In [170]:
van_merged.Neighborhood.loc[van_merged['Cluster Labels'] == 0]

0                Arbutus-Ridge
1                     Downtown
18                      Sunset
17                  Strathcona
4           Grandview-Woodland
5             Hastings-Sunrise
6     Kensington-Cedar Cottage
7                   Kerrisdale
8                    Killarney
16                South Cambie
15                 Shaughnessy
14                  Riley Park
Name: Neighborhood, dtype: object

In [171]:
van_merged.Neighborhood.loc[van_merged['Cluster Labels'] == 1]

21    West Point Grey
11     Mount Pleasant
20           West End
9           Kitsilano
3            Fairview
Name: Neighborhood, dtype: object

In [172]:
van_merged.Neighborhood.loc[van_merged['Cluster Labels'] == 2]

13    Renfrew-Collingwood
2       Dunbar-Southlands
19    Victoria-Fraserview
12               Oakridge
10                Marpole
Name: Neighborhood, dtype: object